# Demo: Considering an easy regression task using the JGepRegression
# Here we start by installing the Julia kernel - this may take a few moments 😴

In [ ]:
%%shell
set +e

#---------------------------------------------------#
JULIA_VERSION="1.10.5" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools CSV DataFrames Dates DynamicExpressions FileIO ForwardDiff GZip JSON LineSearches LinearAlgebra Logging Optim OrderedCollections ProgressMeter Random Serialization StaticArrays Statistics Zygote"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  if ! wget -nv $URL -O /tmp/julia.tar.gz; then
    echo "Failed to download Julia. Check the URL and your internet connection."
    exit 1
  fi

  if ! tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1; then
    echo "Failed to extract Julia archive. Check if you have sufficient permissions."
    exit 1
  fi

  rm /tmp/julia.tar.gz

  # Install packages
  echo "Installing packages..."
  if ! julia -e "using Pkg; Pkg.add([$(echo $JULIA_PACKAGES | sed "s/ /\", \"/g" | sed "s/^/\"/; s/$/\"/")]); Pkg.precompile()"; then
    echo "Failed to install some packages. Please check the output for details."
  fi

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  if ! julia -e 'using Pkg; Pkg.add("IJulia"); using IJulia; IJulia.installkernel("julia", env=Dict("JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'; then
    echo "Failed to install IJulia kernel. Check your internet connection and try again."
    exit 1
  fi

  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  if ! mv -f $KERNEL_NAME "$KERNEL_DIR"/julia; then
    echo "Failed to rename kernel. Check if you have sufficient permissions."
    exit 1
  fi

  echo ''
  echo "Successfully installed Julia $JULIA_VERSION with the specified packages!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "select 'Julia' from the kernel dropdown menu to start using Julia."
else
  echo "Julia is already installed. Version: `julia -v`"
  echo "Updating packages..."
  if ! julia -e "using Pkg; Pkg.add([$(echo $JULIA_PACKAGES | sed "s/ /\", \"/g" | sed "s/^/\"/; s/$/\"/")]); Pkg.update(); Pkg.precompile()"; then
    echo "Failed to update some packages. Please check the output for details."
  fi
fi

Streaming output truncated to the last 5000 lines.
  ◒ CSV
  ◒ ChainRules
  Progress [====================================>    ]  109/122
  ◓ DataFrames
  ◐ CSV
  ◐ ChainRules
  Progress [====================================>    ]  109/122
  ◑ DataFrames
  ◓ CSV
  ◓ ChainRules
  Progress [====================================>    ]  109/122
  ◒ DataFrames
  ◑ CSV
  ◑ ChainRules
  Progress [====================================>    ]  109/122
  ◐ DataFrames
  ◒ CSV
  ◒ ChainRules
  Progress [====================================>    ]  109/122
  ◓ DataFrames
  ◐ CSV
  ◐ ChainRules
  Progress [====================================>    ]  109/122
  ◑ DataFrames
  ◓ CSV
  ◓ ChainRules
  Progress [====================================>    ]  109/122
  ◒ DataFrames
  ◑ CSV
  ◑ ChainRules
  Progress [====================================>    ]  109/122
  ◐ DataFrames
  ◒ CSV
  ◒ ChainRules
  Progress [====================================>    ]  109/122
  ◓ DataFrames
  ◐ CSV
  ◐ ChainRules
  Progre

## After that, go to the right corner (small threefold pointing downwards) and change the runtime type to the julia kernel

## In the nextline we just make sure that we have installed it

In [1]:
versioninfo()

Julia Version 1.10.5
Commit 6f3fdf7b362 (2024-08-27 14:19 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, broadwell)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [2]:
# We install the package: - takes another minute :(
using Pkg
Pkg.add(url="https://github.com/maxreiss123/GEP_SBP_.git")

     Cloning git-repo `https://github.com/maxreiss123/GEP_SBP_.git`
    Updating git-repo `https://github.com/maxreiss123/GEP_SBP_.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [2f0a5bb0] + JGep v0.1.0 `https://github.com/maxreiss123/GEP_SBP_.git#master`
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [2f0a5bb0] + JGep v0.1.0 `https://github.com/maxreiss123/GEP_SBP_.git#master`
Precompiling project...
  ✓ JGep
  1 dependency successfully precompiled in 6 seconds. 130 already precompiled.


In [2]:
#Then we import everthing we need - add further libs if you would like to plot the result
using VGeneExpressionProgramming
using DynamicExpressions
using OrderedCollections
using Random
using CSV
using DataFrames


#If we want to reproduce our results
Random.seed!(1)

TaskLocalRNG()

In [4]:
#Create the utilized symbols: to make the algorithm fast in the backend, we fully tokenize the symbols to Int8 and assign an arity, meaning how many inputs a symbol can have
#The number of the symbols can be chosen arbitrarily, but should match there corresponding representation later on

#Here we use:
#1:=+   which takes 2 arguments
#2:=*   which takes 2 arguments
#3:=/   which takes 2 arguments
#4:=/   which takes 2 arguments
#5:=exp which takes 2 arguments
#
#6 x1   terminal takes 0 arguments
#7 x2   terminal takes 0 arguments
#8 2    terminal takes 0 arguments
#9 0    terminal takes 0 arguments

utilized_syms = OrderedDict{Int8,Int8}(1 => 2, 2 => 2, 3 => 2, 4 => 2, 5 => 1,6 => 0, 7 => 0, 8 => 0, 9 => 0, 10 => 0, 11=> 0)

OrderedDict{Int8, Int8} with 11 entries:
  1  => 2
  2  => 2
  3  => 2
  4  => 2
  5  => 1
  6  => 0
  7  => 0
  8  => 0
  9  => 0
  10 => 0
  11 => 0

In [5]:
#Here we create a vector of symbols serving as the connection between the genes (+,*)
connection_syms = Int8[1, 2, 3, 4]

4-element Vector{Int8}:
 1
 2
 3
 4

In [6]:
#Here, we need to create a mapping between our tokenisation and the symbols utilized for DynamicExpression.jl
#Mapping should corespond to the former defined symbols


operators =  OperatorEnum(; binary_operators=[+, -, *, /], unary_operators=[exp])

callbacks = Dict{Int8,Function}(
        3 => (-),
        4 => (/),
        2 => (*),
        1 => (+),
        5 => (exp)
)

# This time we use 4 nodes according to our dataset, that we use (Feynmann 21.20)
nodes = OrderedDict{Int8,Any}(
    6 => Node{Float64}(feature=1),
    7 => Node{Float64}(feature=2),
    8 => Node{Float64}(feature=3),
    9 => Node{Float64}(feature=4),
    10 => 2,
    11 => 0
)


OrderedDict{Int8, Any} with 6 entries:
  6  => x1
  7  => x2
  8  => x3
  9  => x4
  10 => 2
  11 => 0

In [7]:
#Here we define some hyperparameters for our method


gep_params = Dict{String, AbstractFloat}(
    "one_point_cross_over_prob" => 0.6,
    "two_point_cross_over_prob" => 0.5,
    "mutation_prob" => 1,
    "mutation_rate" => 0.05,
    "dominant_fusion_prob" => 0.1,
    "dominant_fusion_rate" => 0.2,
    "rezessiv_fusion_prob" => 0.1,
    "rezessiv_fusion_rate" => 0.2,
    "fusion_prob" => 0.0,
    "fusion_rate" => 0.0,
    "inversion_prob" => 0.1
)

Dict{String, AbstractFloat} with 11 entries:
  "mutation_rate"             => 0.05
  "dominant_fusion_prob"      => 0.1
  "inversion_prob"            => 0.1
  "dominant_fusion_rate"      => 0.2
  "one_point_cross_over_prob" => 0.6
  "mutation_prob"             => 1.0
  "rezessiv_fusion_rate"      => 0.2
  "fusion_rate"               => 0.0
  "rezessiv_fusion_prob"      => 0.1
  "fusion_prob"               => 0.0
  "two_point_cross_over_prob" => 0.5

## Now lets curl some data 💾



In [12]:
;curl https://raw.githubusercontent.com/maxreiss123/GEP_SBP_/master/test/srsd/feynman-III.21.20%240.txt -o test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1816k  100 1816k    0     0  3233k      0 --:--:-- --:--:-- --:--:-- 3238k


Here 21.20 (https://www.feynmanlectures.caltech.edu/III_21.html) is related to superconductivity, including the Meissner effect. For further information, please follow the link.

$$
\mathbf{J}= -\rho \frac{q}{m} \mathbf{A}
$$


or in later utilized x-notation here:

$$
y= -x3*x2*x1/x4
$$


In [14]:
# Data file, here is expected to be a csv, where the columns are in the order x1,x2...xk, y
data = Matrix(CSV.read("test.txt", DataFrame, header=true))

# Get the number of columns
num_cols = size(data, 2)

# Shuffle the data
data = data[shuffle(1:size(data, 1)), :]

# Split the data into train and test sets
split_point = floor(Int, size(data, 1) * 0.75)
data_train = data[1:split_point, :]
data_test = data[(split_point + 1):end, :]

# Set the consideration factor
consider = 1

# Prepare training data
x_data = Float64.(data_train[1:consider:end, 1:(num_cols-1)])
y_data = Float64.(data_train[1:consider:end, num_cols])

# Prepare test data
x_data_test = Float64.(data_test[1:consider:end, 1:(num_cols-1)])
y_data_test = Float64.(data_test[1:consider:end, num_cols])

5000-element Vector{Float64}:
  -2.5709163186429853
  -5.5235569244901015
  -3.8394696991221866
  -0.9636138832558172
 -24.620051042838757
  -5.441068239825075
 -12.111348083753386
  -5.213480301256804
 -15.754211547031655
 -13.723402334321596
  -5.387788801954733
  -2.5442688753001605
  -1.1852212064528826
   ⋮
  -4.794910526826938
  -1.5572783690853829
  -3.018689387123424
  -3.980661411251999
 -10.58452294521012
 -25.888930012670286
 -13.5831000232206
  -1.0077582208449352
  -8.064940284532904
  -1.8986659138488071
  -7.934338114982501
  -9.428703631883426

In [16]:
#Setting number of individuals
individuals = 1000

#Setting number of epochs
epochs = 1000

#Setting gene count
gene_count = 3

#Setting head len
head_len = 5;


In [18]:
#running the algorithm by using an Mean-squared error
#employing conjugate gradient for the coefficients
#Setting Hall of fame to 1, which means we obtain a list with one element containing the best

best,history=runGep(individuals, epochs,head_len,gene_count,
            utilized_syms,operators, callbacks, nodes, x_data',y_data, connection_syms, gep_params;
    loss_fun_str="mse", opt_method_const=:cg, hof=1);

Progress: 100%|█████████████████████████████████████████████████████████████| Time: 0:00:08


In [19]:
#Showing the fitness and the function
@show string(best[1].fitness)
@show string(best[1].compiled_function)

string((best[1]).fitness) = "1.7069772038870577e-29"
string((best[1]).compiled_function) = "((((((x4 / x3) - x2) - 2.0) + ((2.0 * x4) + x3)) * ((0.0 * x3) / x1)) / (((x3 * x4) + x2) + x2)) - (((x1 * x3) / x4) * x2)"


"((((((x4 / x3) - x2) - 2.0) + ((2.0 * x4) + x3)) * ((0.0 * x3) / x1)) / (((x3 * x4) + x2) + x2)) - (((x1 * x3) / x4) * x2)"

Simplification reveals:
$$
-x1*x2*x3/x4
$$